In [11]:
pip install PyPDF2

In [ ]:
import PyPDF2
import pandas as pd
import re
import os
from openpyxl.utils import get_column_letter
from google.colab import files

In [ ]:
def extract_table_data(pdf_path):
    #open the PDF file
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)

        #store all extracted tables
        all_tables = []
        current_table = []
        table_started = False

        #process each page
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()

            #split text into lines
            lines = text.split('\n')

            #process each line
            for line in lines:
                #skip header and footer lines based on specific patterns
                if re.search(r'BANK NAME|BRANCH NAME|ADDRESS|PIN CODE|STATE|REPORT PRINTED BY|Page No|----------------', line):
                    continue

                #skip account information lines
                if re.search(r'Account No|A/C Name|Address|City|Tel No|A/C Holder|Joint Holders|Open Date|Statement of account|END OF STATEMENT|Grand Total|Manager|Note:|NOTE:|Please examine|If no error', line):
                    continue

                #check if this is the start of account information - could signal end of previous table
                if re.search(r'(Account No)|(A/C Name)', line) and table_started:
                    if current_table:
                        all_tables.append(current_table)
                        current_table = []
                        table_started = False
                    continue

                #check if line contains transaction data
                date_pattern = r'(\d{2}-[A-Za-z]{3}-\d{4}|\d{2}-\d{2}-\d{4}|\d{2}/\d{2}/\d{4})'

                if (re.search(date_pattern, line) or
                     re.search(r'Dr$', line) or
                     re.search(r'\d{2}/\d{2}/\d{4}', line) or
                    (re.search(r'\$', line) and re.search(r'\d+\.\d{2}', line))):

                    #this looks like transaction data
                    if not table_started:
                        table_started = True

                    #extract cells from the line
                    cells = extract_cells_from_line(line)

                    if cells:  #only add non-empty rows
                        current_table.append(cells)

                #check for table end markers
                if re.search(r'Grand Total|END OF STATEMENT', line) and table_started:
                    if current_table:
                        all_tables.append(current_table)
                        current_table = []
                    table_started = False

        #add any remaining table
        if current_table:
            all_tables.append(current_table)

        return all_tables

In [ ]:
def extract_cells_from_line(line):
    date_match = re.search(r'(\d{2}-[A-Za-z]{3}-\d{4}|\d{2}-\d{2}-\d{4}|\d{2}/\d{2}/\d{4})', line)

    if date_match:
        date = date_match.group(1)
        rest_of_line = line[date_match.end():].strip()

        amount_pattern = r'((?:\d{1,3}(?:,\d{3})*|\d+)(?:\.\d{2})?)(?:\s+|$)'

        amounts = re.findall(amount_pattern, rest_of_line)

        description = re.sub(amount_pattern, '', rest_of_line).strip()

        balance = ""
        debit = ""
        credit = ""

        if "Dr" in rest_of_line:
            balance = amounts[-1] + " Dr" if amounts else ""
            amounts = amounts[:-1] if amounts else []

            if len(amounts) >= 1:
                if any(credit_marker in description for credit_marker in ["C By Cash", "Cr-IMPS", "Credit", "UPI/CR"]):
                    credit = amounts[0]
                else:
                    debit = amounts[0]

        else:
            if len(amounts) >= 3:
                debit = amounts[0] if amounts[0] else ""
                credit = amounts[1] if amounts[1] else ""
                balance = amounts[2] if amounts[2] else ""
            elif len(amounts) == 2:
                if any(credit_marker in description for credit_marker in ["C By", "CR", "Credit", "CREDIT"]):
                    credit = amounts[0] if amounts[0] else ""
                    balance = amounts[1] if amounts[1] else ""
                else:
                    debit = amounts[0] if amounts[0] else ""
                    balance = amounts[1] if amounts[1] else ""
            elif len(amounts) == 1:
                balance = amounts[0]

        row = [date, description, debit, credit, balance]
        return row

    elif re.search(r'\d+\.\d{2}', line):  #has a decimal amount
        amount_pattern = r'((?:\d{1,3}(?:,\d{3})*|\d+)(?:\.\d{2})?)(?:\s+|$)'
        amounts = re.findall(amount_pattern, line)

        description = re.sub(amount_pattern, '', line).strip()

        debit = ""
        credit = ""
        balance = ""

        if len(amounts) >= 3:
            debit = amounts[0]
            credit = amounts[1]
            balance = amounts[2]
        elif len(amounts) == 2:
            if any(credit_marker in description for credit_marker in ["C By", "CR", "Credit", "CREDIT"]):
                credit = amounts[0]
                balance = amounts[1]
            else:
                debit = amounts[0]
                balance = amounts[1]
        elif len(amounts) == 1:
            balance = amounts[0]

        row = ["", description, debit, credit, balance]
        return row

    return None

In [ ]:
def save_to_excel(tables, output_path):
    #create a new Excel file
    writer = pd.ExcelWriter(output_path, engine='openpyxl')

    #process each table
    for i, table in enumerate(tables):
        headers = ["Date", "Description", "Debit", "Credit", "Balance"]

        #create DataFrame
        df = pd.DataFrame(table, columns=headers)

        #write DataFrame to Excel
        sheet_name = f"Table_{i+1}"
        df.to_excel(writer, sheet_name=sheet_name, index=False)

        #auto-adjust column widths
        worksheet = writer.sheets[sheet_name]
        for idx, col in enumerate(df.columns):
            max_len = max(df[col].astype(str).map(len).max(), len(col)) + 2
            worksheet.column_dimensions[get_column_letter(idx+1)].width = max_len

    #save Excel file
    writer.close()
    print(f"Tables saved to {output_path}")

In [ ]:
def process_pdf(pdf_path, output_dir=None):
    #get base filename without extension
    base_filename = os.path.splitext(os.path.basename(pdf_path))[0]

    #determine output directory
    if not output_dir:
        output_dir = os.path.dirname(pdf_path)

    #create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    #set output path
    output_path = os.path.join(output_dir, f"{base_filename}_tables.xlsx")

    #extract tables
    print(f"Extracting tables from {pdf_path}...")
    tables = extract_table_data(pdf_path)

    if tables:
        #save tables to Excel
        save_to_excel(tables, output_path)
        print(f"Successfully extracted {len(tables)} table(s) from {pdf_path}")
        return output_path
    else:
        print(f"No tables found in {pdf_path}")
        return None

In [ ]:
def upload_and_process_pdf():
    #prompt user to upload PDF
    uploaded = files.upload()

    for filename, content in uploaded.items():
        if filename.lower().endswith('.pdf'):
            #save the uploaded file
            with open(filename, 'wb') as f:
                f.write(content)

            #process the PDF file
            output_path = process_pdf(filename, './output')

            #download the output Excel file
            if output_path and os.path.exists(output_path):
                files.download(output_path)
        else:
            print(f"Skipping {filename} - not a PDF file")

In [13]:
upload_and_process_pdf()

Saving test3.pdf to test3 (2).pdf
Extracting tables from test3 (2).pdf...
Tables saved to ./output/test3 (2)_tables.xlsx
Successfully extracted 1 table(s) from test3 (2).pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>